In [2]:
from google.colab import files
import pandas as pd
uploaded = files.upload()
df = pd.read_csv("1faculty_data.csv")
df.head()

Saving 1faculty_data.csv to 1faculty_data (1).csv


,EmpId,Department,Teaching,Lenient,DA and Quiz,Marking,Overall
0,10527,SAS,7.5,6.8,8.2,7.1,7.4
1,15752,SAS,8.3,5.9,7.5,6.8,7.1
2,21772,SAS,6.9,7.2,6.5,7.8,7.1
3,10561,SAS,8.1,6.3,7.9,7.5,7.5
4,13281,SAS,7.8,8.2,7.0,7.6,7.7


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
encoder = OneHotEncoder(sparse_output=False)
dept_encoded = encoder.fit_transform(df[['Department']])
dept_columns = encoder.get_feature_names_out(['Department'])
df_dept = pd.DataFrame(dept_encoded, columns=dept_columns)
df = pd.concat([df.drop('Department', axis=1), df_dept], axis=1)
X = df.drop(columns=['EmpId'])
y = df[['Teaching', 'Lenient', 'DA and Quiz', 'Marking']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("✅ Data Preprocessing Complete")

✅ Data Preprocessing Complete


In [4]:
from sklearn.ensemble import RandomForestRegressor
import pickle
import os
from google.colab import drive
models = {}
categories = ['Teaching', 'Lenient', 'DA and Quiz', 'Marking']
for category in categories:
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[category])
    models[category] = model
print("✅ Model Training Complete")
drive.mount('/content/drive')
model_path = '/content/drive/My Drive/FFCS_Helper_Models/'
os.makedirs(model_path, exist_ok=True)
for category, model in models.items():
    with open(os.path.join(model_path, f"{category}_model.pkl"), 'wb') as file:
        pickle.dump(model, file)
with open(os.path.join(model_path, 'scaler.pkl'), 'wb') as file:
    pickle.dump(scaler, file)
print(f"✅ Models & Scaler Saved Successfully in Google Drive: {model_path}")

✅ Model Training Complete
Mounted at /content/drive
✅ Models & Scaler Saved Successfully in Google Drive: /content/drive/My Drive/FFCS_Helper_Models/


In [5]:
models = {}
for category in categories:
    with open(os.path.join(model_path, f"{category}_model.pkl"), 'rb') as file:
        models[category] = pickle.load(file)

with open(os.path.join(model_path, 'scaler.pkl'), 'rb') as file:
    scaler = pickle.load(file)

print("✅ Models Loaded Successfully")
def predict_best_faculty(faculty_list, student_prefs):
    """
    Predicts and ranks faculties based on different rating categories and student preferences.
    Returns:
    - Best faculty for each category (Teaching, Lenient, DA and Quiz, Marking).
    - Overall best faculty based on student preferences.
    - Full ranked list of faculties.
    """
    filtered_faculty = df[df['EmpId'].isin(faculty_list)].copy()

    if filtered_faculty.empty:
        return {"message": "Prediction not available. No faculty found in the given list."}
    faculty_features = filtered_faculty.drop(columns=['EmpId'])
    faculty_features_scaled = scaler.transform(faculty_features)
    predicted_scores = {cat: models[cat].predict(faculty_features_scaled) for cat in categories}
    for cat in categories:
        filtered_faculty[f"Predicted_{cat}"] = predicted_scores[cat]
    best_faculty_per_category = {
        category: filtered_faculty.loc[filtered_faculty[f"Predicted_{category}"].idxmax(), ['EmpId', f"Predicted_{category}"]].to_dict()
        for category in categories
    }
    weight_vector = np.array([student_prefs.get(feature, 1) for feature in categories])
    weight_vector = weight_vector / weight_vector.sum()
    faculty_characteristics = filtered_faculty[[f"Predicted_{cat}" for cat in categories]].values
    custom_scores = np.dot(faculty_characteristics, weight_vector)
    filtered_faculty["Custom_Score"] = custom_scores
    ranked_faculty = filtered_faculty.sort_values(by="Custom_Score", ascending=False)
    best_overall_faculty = ranked_faculty.iloc[0][['EmpId', 'Custom_Score']].to_dict()

    return {
        "best_per_category": best_faculty_per_category,
        "overall_best_faculty": best_overall_faculty,
        "ranked_faculties": ranked_faculty[['EmpId', 'Custom_Score']].to_dict(orient='records')
    }

faculty_list = [10527, 15752, 21772]  #Sample Input Data
student_prefs = {
    'Teaching': 8.0,
    'Lenient': 6.5,
    'DA and Quiz': 7.0,
    'Marking': 7.5
}
result = predict_best_faculty(faculty_list, student_prefs)
print("✅ Best Faculty Per Category:")
for category, faculty in result["best_per_category"].items():
    print(f"{category}: Faculty {faculty['EmpId']} (Score: {faculty[f'Predicted_{category}']:.2f})")
print("\n🏆 Overall Best Faculty:")
print(f"Faculty {result['overall_best_faculty']['EmpId']} (Custom Score: {result['overall_best_faculty']['Custom_Score']:.2f})")
print("\n📜 Full Faculty Ranking:")
for faculty in result["ranked_faculties"]:
    print(f"Faculty {faculty['EmpId']} - Score: {faculty['Custom_Score']:.2f}")
faculty_list = [10527, 15752, 21772]
student_prefs = {
    'Teaching': 8.0,
    'Lenient': 10,
    'DA and Quiz': 9,
    'Marking': 0.5
}
result = predict_best_faculty(faculty_list, student_prefs)
print(result)


✅ Models Loaded Successfully
✅ Best Faculty Per Category:
Teaching: Faculty 15752.0 (Score: 8.23)
Lenient: Faculty 21772.0 (Score: 7.28)
DA and Quiz: Faculty 10527.0 (Score: 8.09)
Marking: Faculty 21772.0 (Score: 7.82)

🏆 Overall Best Faculty:
Faculty 10527.0 (Custom Score: 7.38)

📜 Full Faculty Ranking:
Faculty 10527 - Score: 7.38
Faculty 15752 - Score: 7.18
Faculty 21772 - Score: 7.14
{'best_per_category': {'Teaching': {'EmpId': 15752.0, 'Predicted_Teaching': 8.233999999999988}, 'Lenient': {'EmpId': 21772.0, 'Predicted_Lenient': 7.276000000000002}, 'DA and Quiz': {'EmpId': 10527.0, 'Predicted_DA and Quiz': 8.092000000000008}, 'Marking': {'EmpId': 21772.0, 'Predicted_Marking': 7.817999999999994}}, 'overall_best_faculty': {'EmpId': 10527.0, 'Custom_Score': 7.424418181818183}, 'ranked_faculties': [{'EmpId': 10527, 'Custom_Score': 7.424418181818183}, {'EmpId': 15752, 'Custom_Score': 7.153509090909084}, {'EmpId': 21772, 'Custom_Score': 6.954909090909091}]}
